## MAPA MUNDI 

# lo que se hizo
- Se realizaron los graficos del pais, con botones por su bacteria corespondiente 

In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp

escalaColor='viridis'

In [2]:
df = pd.read_csv('datos_taxonomiaCompleta2.csv')

In [3]:
df.columns

Index(['Accession', 'Description', 'Taxonomy', 'Specie', 'Strain', 'Organism',
       'Modification Date', 'Country', 'Isolation Source', 'Host', 'Sequence',
       'Class', 'Localidad', 'Longitud', 'Latitud', 'Longitud secuencia',
       'Taxonomy:superkingdom', 'Taxonomy:phylum', 'Taxonomy:class',
       'Taxonomy:order', 'Taxonomy:family', 'Taxonomy:genus',
       'Taxonomy:species'],
      dtype='object')

In [4]:
noEspecificado=df[df['Country']!='No especificado']
paisesGrafico, indice, frecuenciaPaises=np.unique(noEspecificado['Country'], return_counts=True,return_index=True)
totalNoEspecificado=len(df[df['Country']=='No especificado'])

In [5]:
dfgrafico=noEspecificado.iloc[indice]

In [6]:
latitud=noEspecificado[noEspecificado['Latitud']!=None]
longitud=noEspecificado[noEspecificado['Longitud']!=None]

In [7]:
datosGraficoGlobal=[paisesGrafico, frecuenciaPaises, totalNoEspecificado, f'Total bacterias <br>Sin localidad{totalNoEspecificado}']

In [8]:
# Crear un array de 9x9 con todos los elementos a False
array = np.full((10, 10), False)

# Establecer True en la diagonal
np.fill_diagonal(array, True)

In [9]:
clasesBacteriasCarpetas = ['Abiotrophia','Acidovorax', 'Cyanibacteria', 'Megasphaera','Mycobacterium','Neisseria','Prevotella', 'Streptococcus','Veillonella']
datosBacteriasGlobales=[datosGraficoGlobal]
botones=[dict(label=f'Datos globales',
                      method='update',
                        args=[{'visible': array[0]}, {'title': f'Datos sin especificar del total del muestreo: {totalNoEspecificado}'}]
                    )]
for i in range(len(clasesBacteriasCarpetas)):
    tempBac=df[df['Class']==clasesBacteriasCarpetas[i]]
    largoNE=len(tempBac[tempBac['Country']=='No especificado'])
    tempBac=tempBac[tempBac['Country']!='No especificado']
    tempPaisesBacteria, tempConteo=np.unique(tempBac['Country'], return_counts=True)
    datosBacteriasGlobales.append([tempPaisesBacteria, tempConteo,largoNE, f'Bacteria: {clasesBacteriasCarpetas[i]} <br>Sin localidad: {largoNE}'])
    botones.append(dict(
                        label=f'Bacteria {clasesBacteriasCarpetas[i]}',
                        method='update',
                        args=[{'visible': array[i+1]}, {'title': f'Bacteria: {clasesBacteriasCarpetas[i]} <br>Sin localidad: {largoNE}'}]
                    )
    )
    print(clasesBacteriasCarpetas[i],len(tempBac), largoNE)

Abiotrophia 41 430
Acidovorax 1220 2975
Cyanibacteria 32 36
Megasphaera 293 564
Mycobacterium 4161 5760
Neisseria 864 3311
Prevotella 948 9051
Streptococcus 3597 6372
Veillonella 464 4478


In [15]:
def graficoBacteria(i):
    fig =go.Figure()
    fig.add_trace(go.Choropleth(
        locations=i[0],
        locationmode='country names',
        z=np.log(i[1]),
        hovertemplate =['Pais: <b>%s</b><br>Frec.Bacterias : <b>%d</b>' % (a, b) for a, b in zip(i[0], i[1])],
        colorscale=escalaColor,
        reversescale=True,
        colorbar=dict(
            title='Escala de colores',
            tickvals= np.linspace(min(np.log(i[1]).astype(int).tolist()), max(np.log(i[1]).astype(int).tolist()),10), #[min(np.log(frecuenciaPaises).astype(int).tolist()), max(np.log(frecuenciaPaises).astype(int).tolist())],
            ticktext=np.linspace(min(i[1].astype(int).tolist()), max(i[1]).astype(int).tolist(),10).astype(int),#['Mínimo', 'Máximo'],
            ticks='outside',
        ),
        
    ))
    fig.update_layout(
            geo=dict(
                showland=True,
                landcolor='rgb(217, 217, 217)',
                showcountries=True,
                showcoastlines=True,
                projection_type='natural earth'
            ),
            title=i[3]
    )
    
    fig.update_geos(projection_type="natural earth")
    fig.update_layout(width=1200, height=600)
    # Mostrar el gráfico
    fig.show()




<style>body {text-align: right}</style>

# Titulo


In [16]:
# visualizacion de los graficos 0,1,2...
# globales, 
graficoBacteria(datosBacteriasGlobales[0])

In [17]:
#conversion de string fecha de modificacion 
from datetime import datetime
formato_fecha = '%d-%b-%Y'
tempfechas=[]
for i in df['Modification Date']:
    tempfechas.append(datetime.strptime(i, formato_fecha))

In [18]:
dfCopia=df.copy()
dfCopia['Modification Date'] = tempfechas

In [19]:
tempaño=[]
for i in range(len(dfCopia)):
    tempaño.append(dfCopia['Modification Date'][i].year)

In [20]:
dfCopia['Modification Date']=tempaño

In [21]:
dfCopia.sort_values(by='Modification Date', inplace=True)

In [22]:
#tempNE= temp no especificado
#tempPG= temp paises grafico
#tempI= temp indice
#tempFP= temp frecuencia paises

def procesamientoDatosBacterias():
    dfs=[]
    for i in range(len(clasesBacteriasCarpetas)+1):
        if i ==0:
            #tempNE=df[df['Country']!='No especificado']
            #tempPG, tempI, tempFP=np.unique(noEspecificado['Country'], return_counts=True,return_index=True)
            #totalNoEspecificado=len(df[df['Country']=='No especificado'])
            tempdf=dfCopia[['Country', 'Modification Date', 'Class']]
            dfs.append(tempdf)
        else: 
            tempdf=dfCopia[dfCopia['Class']==clasesBacteriasCarpetas[i-1]]
            tempdf=tempdf[['Country', 'Modification Date', 'Class']]
            dfs.append(tempdf)
        
    return dfs
        
    

In [23]:
dataFramesGrafico=procesamientoDatosBacterias()

In [24]:
dataFramesGrafico[1]

,Country,Modification Date,Class
247,No especificado,1997,Abiotrophia
306,No especificado,1999,Abiotrophia
393,No especificado,1999,Abiotrophia
346,No especificado,1999,Abiotrophia
254,No especificado,1999,Abiotrophia
...,...,...,...
374,No especificado,2023,Abiotrophia
419,No especificado,2023,Abiotrophia
259,Argentina,2023,Abiotrophia
440,No especificado,2023,Abiotrophia


In [25]:
dataFramesGrafico[0]['Modification Date'][1]

2010

In [26]:
xdPG, xdI, xdFP=np.unique(dataFramesGrafico[1]['Modification Date'], return_counts=True,return_index=True)


In [27]:
dftemp={'fechas':xdFP,
        'frecuenciaPaises':xdPG,
        }
dftemp=pd.DataFrame(dftemp)


In [62]:
#No Especificado DfCopy
nedc=dataFramesGrafico[0][dataFramesGrafico[0]['Country']!='No especificado']
nedc[nedc['Class']==clasesBacteriasCarpetas[1]]

,Country,Modification Date,Class
1406,Germany,2001,Acidovorax
3424,Canada,2002,Acidovorax
3571,South Korea,2002,Acidovorax
1111,France,2003,Acidovorax
624,United States of America,2003,Acidovorax
...,...,...,...
2645,China,2023,Acidovorax
930,Russia,2023,Acidovorax
2484,China,2023,Acidovorax
2472,China,2023,Acidovorax


In [61]:
dataFramesGrafico[0]

,Country,Modification Date,Class
7769,No especificado,1991,Mycobacterium
14933,No especificado,1991,Mycobacterium
15830,No especificado,1991,Neisseria
14244,No especificado,1991,Mycobacterium
6876,No especificado,1991,Mycobacterium
...,...,...,...
5634,Mexico,2023,Mycobacterium
29063,Mexico,2023,Prevotella
1215,No especificado,2023,Acidovorax
35706,Iraq,2023,Streptococcus


In [29]:
nedc['Modification Date'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023], dtype=int64)

In [30]:
np.full(10,2000)

array([2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000])

In [31]:
tempfa=[]
for i in nedc['Modification Date'].unique():
    temp1=nedc[nedc['Modification Date']==i]
    temp2, temp3= np.unique(temp1['Country'], return_counts=True)
    temp4=np.full(len(temp2), i) # crea una lista de NxM y la rellena con un parametro
    dftemp=pd.DataFrame({'año': temp4,
                         'pais': temp2,
                         'frecuenciaPais':temp3})
    tempfa.append(dftemp)

In [76]:
#clasesBacteriasCarpetas
def rsGraficoAños():
    datosrs=[]
    datosrsne=[]
    for j in clasesBacteriasCarpetas:
        tempfa=[]
        dfrs=dataFramesGrafico[0][dataFramesGrafico[0]['Class']==j]
        dfrs=dfrs[dfrs['Country']!='No especificado']
        tempne=len(dfrs[dfrs['Country']=='No especificado'])
        datosrsne.append(tempne)
        for i in dfrs['Modification Date'].unique():
            temp1=dfrs[dfrs['Modification Date']==i]
            temp2, temp3= np.unique(temp1['Country'], return_counts=True)
            temp4=np.full(len(temp2), i) # crea una lista de NxM y la rellena con un parametro
            dftemp=pd.DataFrame({'año': temp4,
                                'pais': temp2,
                                'frecuenciaPais':temp3})
            tempfa.append(dftemp)
        datosrs.append(tempfa)
    return datosrs, datosrsne


In [77]:
datosRangeSlider, datosRangeSliderNE=rsGraficoAños()

In [78]:
datosRangeSlider[0][0]

,año,pais,frecuenciaPais
0,2003,United Kingdom,1


In [79]:
tempfa[1]

,año,pais,frecuenciaPais
0,2001,Denmark,1
1,2001,Finland,3
2,2001,Germany,1
3,2001,Malawi,22
4,2001,Spain,1
5,2001,United Kingdom,1
6,2001,United States of America,1


In [83]:
def graficoRS(tempfa):    
    añoMinimo=min(tempfa[0]['año'])
    dataslider=[]
    for i in tempfa:
        aux=i['año'][0]
        dataPorAño=dict(
            type='choropleth',
            locations=i['pais'],
            z=i['frecuenciaPais'].astype(int),
            locationmode='country names',
            colorscale=escalaColor,
            reversescale=True,
            colorbar={'title':f'N° bacterias{aux}'})
        dataslider.append(dataPorAño)

    pasos=[]
    for i in range(len(dataslider)):
        step = dict(method='restyle',
                    args=['visible', [False] * len(dataslider)],
                    label='Year {}'.format(i + añoMinimo))
        step['args'][1][i] = True
        pasos.append(step)
    sliders = [dict(active=0, pad={"t": 1}, steps=pasos)]

    layout = dict(title ='Cantidad de bacterias por año', geo=dict(
                        projection={'type': 'natural earth'}),
                sliders=sliders,
                )

    fig = dict(data=dataslider, layout=layout)
    fig=go.Figure(fig)
    fig.update_layout(geo=dict(
                    showland=True,
                    landcolor='rgb(217, 217, 217)',
                    showcountries=True,
                    showcoastlines=True,
                    projection_type='natural earth'))
    fig.update_layout(width=1200, height=600)
    fig.show()
graficoRS(datosRangeSlider[0])

In [34]:
np.array(df[['Country', 'Class']])

array([['No especificado', 'Abiotrophia'],
       ['No especificado', 'Abiotrophia'],
       ['No especificado', 'Abiotrophia'],
       ...,
       ['No especificado', 'Veillonella'],
       ['No especificado', 'Veillonella'],
       ['No especificado', 'Veillonella']], dtype=object)

In [35]:
dfCopia[dfCopia['Modification Date']==1997]['Country']

12769    No especificado
12531    No especificado
247      No especificado
6187     No especificado
14729    No especificado
6641     No especificado
11477    No especificado
15176    No especificado
12386    No especificado
13126    No especificado
7104     No especificado
6959     No especificado
7889     No especificado
Name: Country, dtype: object

In [36]:
dataFramesGrafico[4]

,Country,Modification Date,Class
5061,No especificado,1996,Megasphaera
5265,No especificado,2000,Megasphaera
5011,No especificado,2000,Megasphaera
5327,No especificado,2000,Megasphaera
5016,No especificado,2002,Megasphaera
...,...,...,...
5400,No especificado,2023,Megasphaera
5323,No especificado,2023,Megasphaera
5366,No especificado,2023,Megasphaera
5578,No especificado,2023,Megasphaera


In [37]:
df['Modification Date']= tempfechas

In [38]:
datosfigures=[]
for i in datosBacteriasGlobales:
    datosfigures.append(graficoBacteria(i))


In [41]:
# fig = go.Figure(data=datosfigures)
# layout = go.Layout(
#     geo=dict(
#         showland=True,
#         landcolor='rgb(217, 217, 217)',
#         showcountries=True,
#         showcoastlines=True,
#         projection_type='natural earth'
#     )
# )
# for i in datosBacteriasGlobales:
#     fig.add_trace(go.Choropleth(
#         locations=i[0],
#         locationmode='country names',
#         z=np.log(i[1]),
#         hovertemplate =['Pais: <b>%s</b><br>Frec.Bacterias : <b>%d</b>' % (a, b) for a, b in zip(i[0], i[1])],
#         colorscale=escalaColor,
#         reversescale=True,
#         colorbar=dict(
#             title='Escala de colores',
#             tickvals= np.linspace(min(np.log(i[1]).astype(int).tolist()), max(np.log(i[1]).astype(int).tolist()),10), #[min(np.log(frecuenciaPaises).astype(int).tolist()), max(np.log(frecuenciaPaises).astype(int).tolist())],
#             ticktext=np.linspace(min(i[1].astype(int).tolist()), max(i[1]).astype(int).tolist(),10).astype(int),#['Mínimo', 'Máximo'],
#             ticks='outside',
#         ),
        
#     ))
#     fig.update_layout(
#             geo=dict(
#                 showland=True,
#                 landcolor='rgb(217, 217, 217)',
#                 showcountries=True,
#                 showcoastlines=True,
#                 projection_type='natural earth'
#             )
#     )

# Actualizar el diseño de la figura
# fig.update_layout(
#     updatemenus=[
#         dict(
#             type='buttons',
#             showactive=True,
#             direction='down',
#             buttons=botones,
#             active=4)]
# )


# fig.update_geos(projection_type="natural earth")
# fig.update_layout(width=1200, height=600)
# # Mostrar el gráfico
# fig.show()

In [42]:
fig = go.Figure()
layout = go.Layout(
    geo=dict(
        showland=True,
        landcolor='rgb(217, 217, 217)',
        showcountries=True,
        showcoastlines=True,
        projection_type='natural earth'
    )
)
for i in datosBacteriasGlobales:
    fig.add_trace(go.Choropleth(
        locations=i[0],
        locationmode='country names',
        z=np.log(i[1]),
        hovertemplate =['Pais: <b>%s</b><br>Frec.Bacterias : <b>%d</b>' % (a, b) for a, b in zip(i[0], i[1])],
        colorscale=escalaColor,
        reversescale=True,
        colorbar=dict(
            title='Escala de colores',
            tickvals= np.linspace(min(np.log(i[1]).astype(int).tolist()), max(np.log(i[1]).astype(int).tolist()),10), #[min(np.log(frecuenciaPaises).astype(int).tolist()), max(np.log(frecuenciaPaises).astype(int).tolist())],
            ticktext=np.linspace(min(i[1].astype(int).tolist()), max(i[1]).astype(int).tolist(),10).astype(int),#['Mínimo', 'Máximo'],
            ticks='outside',
        ),
        
    ))
    fig.update_layout(
            geo=dict(
                showland=True,
                landcolor='rgb(217, 217, 217)',
                showcountries=True,
                showcoastlines=True,
                projection_type='natural earth'
            )
    )

# Actualizar el diseño de la figura
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            showactive=True,
            direction='down',
            buttons=botones,
            active=4)]
)

fig.update_geos(projection_type="natural earth")
fig.update_layout(width=1200, height=600)
# Mostrar el gráfico
fig.show()

In [43]:
choropleth = go.Choropleth(
    locations=paisesGrafico,
    locationmode='country names',
    z=np.log(frecuenciaPaises),
    hovertemplate =['Pais: <b>%s</b><br>Frec.Bacterias : <b>%d</b>' % (a, b) for a, b in zip(paisesGrafico, frecuenciaPaises)],
    colorscale=escalaColor,
    reversescale=True,
    colorbar=dict(
        title='Escala de colores',
        tickvals= np.linspace(min(np.log(frecuenciaPaises).astype(int).tolist()), max(np.log(frecuenciaPaises).astype(int).tolist()),10),
        ticktext=np.linspace(min(frecuenciaPaises.astype(int).tolist()), max(frecuenciaPaises).astype(int).tolist(),10).astype(int),
        ticks='outside',
    ),
)

layout = go.Layout(
    geo=dict(
        showland=True,
        landcolor='rgb(217, 217, 217)',
        showcountries=True,
        showcoastlines=True,
        projection_type='natural earth'
    ),
        annotations=[
        dict(
            x=0,
            y=-0.15,
            xref='paper',
            yref='paper',
            text=f'Total No especificado: {totalNoEspecificado}',
            showarrow=False,
            font=dict(size=12),
        ),
    ]
)

fig = go.Figure(data=[choropleth], layout=layout)
fig.update_geos(projection_type="natural earth")
fig.update_layout(width=1200, height=600)
fig.show()

In [ ]:
superkingdom=np.array(df['Taxonomy:superkingdom'])
phylum=np.array(df['Taxonomy:phylum'])
classes=np.array(df['Taxonomy:class'])
order=np.array(df['Taxonomy:order'])
family=np.array(df['Taxonomy:family'])
genus=np.array(df['Taxonomy:genus'])
species=np.array(df['Taxonomy:species'])


In [57]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# def funcion1(b):
#     with output:
#         clear_output(wait=True)
#         graficoBacteria(datosBacteriasGlobales[0])

# def funcion2(b):
#     with output:
#         clear_output(wait=True)
#         graficoBacteria(datosBacteriasGlobales[1])

# def funcion3(b):
#     with output:
#         clear_output(wait=True)
#         graficoBacteria(datosBacteriasGlobales[2])
def funcion1(b):
    with output:
        if b == ultimo_boton:
            clear_output(wait=True)
        graficoBacteria(datosBacteriasGlobales[0])

def funcion2(b):
    with output:
        if b == ultimo_boton:
            clear_output(wait=True)
        graficoBacteria(datosBacteriasGlobales[1])

def funcion3(b):
    with output:
        if b == ultimo_boton:
            clear_output(wait=True)
        graficoBacteria(datosBacteriasGlobales[2])


In [58]:
# Crear botones
boton1 = widgets.Button(description="Función 1")
boton2 = widgets.Button(description="Función 2")
boton3 = widgets.Button(description="Función 3")

# Asignar funciones a los botones
boton1.on_click(funcion1)
boton2.on_click(funcion2)
boton3.on_click(funcion3)


In [60]:
# Crear un área de salida
output = widgets.Output()

# Almacenar el último botón presionado
ultimo_boton = None

# Función para actualizar el último botón presionado
def actualizar_ultimo_boton(b):
    global ultimo_boton
    ultimo_boton = b

# Asignar la función de actualización al evento on_click de cada botón
boton1.on_click(actualizar_ultimo_boton)
boton2.on_click(actualizar_ultimo_boton)
boton3.on_click(actualizar_ultimo_boton)

# Organizar los botones horizontalmente
botones_horizontal = widgets.HBox([boton1, boton2, boton3])

# Mostrar los botones y el área de salida
display(botones_horizontal, output)

Output()